In [ ]:
import numpy as np
import torch as torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import copy
from saga import SAGA
from svrg import SVRG
from torch.optim.optimizer import Optimizer, required
from sklearn.linear_model import LinearRegression
from torch.optim.lr_scheduler import StepLR,LambdaLR
from helpers import *
import time
import math
from resnet import ResNet18
%matplotlib inline

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = "cpu"

In [ ]:
N_CLUSTERS = 1
MOMENTUM_SAGA = 0.01
MOMENTUM_SGD = 0.9
N_SAMPLES = 50000
betas = (0.9,0.999)
BATCH_SIZE = 1
data = "cifar10"
X,y,X_test,y_test,IN_DIM,OUT_DIM = get_data("cifar10",3,0)
lr = 0.0001
n_epochs = 500000
res_step = int(n_epochs/100) #compute loss,variance and error every step iteration /100 -> 100 values
#res_step_var = int(n_epochs/100)
opti = "Adam"
centered = "uncentered"
if (centered == "centered"):
    X = X/127.5-1
    X_test = X_test/127.5-1
else:
    X = X/255
    X_test = X_test/255

In [ ]:
print(X.shape)
print(X_test.shape)

In [ ]:
print(X.mean())
print(X_test.mean())

In [ ]:
print(X.min())
print(X_test.min())

In [ ]:
print(X.max())
print(X_test.max())

In [ ]:
class CNN(nn.Module):
    def __init__(self,out_dim):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, out_dim)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# X_cluster,y_cluster,_,_,_,_ = get_data(data,1,0,greyscale = True)

# def get_data_partitions():
#     Xs = []
#     for i in range(OUT_DIM):
#         Xs.append(X_cluster[np.argwhere(y==i)])
#     return tuple(Xs)

# Xs = get_data_partitions()

# from sklearn.cluster import KMeans
# #number of cluster per class and list of data, each only containing a single example
# def get_class_clusters(n_clusters,data_partitions):
#     kmeans = []
#     for i in range(len(data_partitions)):
#         kmeans.append(KMeans(n_clusters = n_clusters,random_state=0).fit(data_partitions[i].squeeze()))
#     return kmeans

# class_clusters = get_class_clusters(N_CLUSTERS*4,get_data_partitions())

# def get_cc_proba(class_clusters,N_CLUSTERS):
#     c = np.zeros((OUT_DIM*N_CLUSTERS))
#     for i in range(OUT_DIM):
#         for d in Xs[i]:
#             c[N_CLUSTERS*i + class_clusters[i].predict(d).item()] +=1
#     print(c)
#     return c/X.shape[0]

# c_prob = get_cc_proba(class_clusters,N_CLUSTERS*4)
# print(c_prob)

In [ ]:
model_SGD = ResNet18(OUT_DIM).to(device)
# model_SAGA = ResNet18(OUT_DIM).to(device)

model_SGD_losses = []
model_SGD_epoch_losses = []
SGD_avg_var = []
SGD_epoch_var = []
model_SGD_test_e = []

model_SAGA_losses = []
model_SAGA_epoch_losses = []
SAGA_avg_var = []
SAGA_epoch_var = []
model_SAGA_test_e = []

# model_SAGA_2 = ResNet18(OUT_DIM).to(device)
# model_SAGA_2_losses = []
# model_SAGA_2_epoch_losses = []
# SAGA_2_avg_2_var = []
# SAGA_epoch_2_var = []
# model_SAGA_2_test_e = []



criterion = nn.CrossEntropyLoss()

optimizer_SGD = torch.optim.Adam(model_SGD.parameters(), lr = lr, betas = betas)
# optimizer_SAGA = SAGA(model_SAGA.parameters(), n_classes=OUT_DIM, lr = lr,
#                           class_proba = None,momentum=MOMENTUM_SAGA,compute_var = True,betas = betas,
#                          use_adam = True)
#NO MOMENTUM
# optimizer_SAGA = SAGA(model_SAGA.parameters(), n_classes=OUT_DIM, lr = lr,
#                           class_proba = None,momentum=MOMENTUM_SAGA,compute_var = True,betas = betas,
#                          use_adam = True)


# model_SAGA_2 = ResNet18(OUT_DIM).to(device)
model_SAGA_2_losses = []
model_SAGA_2_epoch_losses = []
SAGA_2_avg_var = []
SAGA_2_epoch_var = []
model_SAGA_2_test_e = []
# optimizer_SAGA_2 = SAGA(model_SAGA_2.parameters(), n_classes=OUT_DIM*4, lr = lr,
#                           class_proba = c_prob,momentum=MOMENTUM_SAGA,compute_var = True,betas = betas,
#                          use_adam = True)


model_SVRG =  ResNet18(OUT_DIM).to(device)
model_SVRG_losses = []
model_SVRG_epoch_losses = []
SVRG_avg_var = []
SVRG_epoch_var = []
model_SVRG_test_e = []
optimizer_SVRG = SVRG(model_SVRG.parameters(), lr = lr,compute_var = True,
                                use_adam = True)

lr_lambda = lambda epoch : 1/np.sqrt(epoch+1)
scheduler_SGD = LambdaLR(optimizer_SGD, lr_lambda = lr_lambda)
# scheduler_SAGA = LambdaLR(optimizer_SAGA, lr_lambda = lr_lambda)
# scheduler_SAGA_2 = LambdaLR(optimizer_SAGA_2, lr_lambda = lr_lambda)
scheduler_SVRG = LambdaLR(optimizer_SVRG, lr_lambda = lr_lambda)

In [ ]:
def compute_error(model,X,y):
    e = 0
    batch_size = 200
    for i in range(int(X.shape[0]/batch_size)):
        pred = model.forward(torch.Tensor(X_test[i*batch_size:(i+1)*batch_size]).to(device))
        pred = (torch.max(pred.data,1)[1]).cpu().numpy()
        err = (y[i*batch_size:(i+1)*batch_size] != pred)
        e += err.sum()
    return e/X.shape[0]

In [ ]:
# compute_error(model_SAGA,X_test,y_test)

In [ ]:
# # train SGD
# tstart = time.time()
# total_var = 0
# epoch_var = 0
# total_loss = 0
# epoch_loss = 0
# for epoch in range(n_epochs):
#     #epoch loss and var
#     if ((epoch % res_step == 0) and (epoch != 0)):
#         model_SGD_epoch_losses.append(epoch_loss/res_step)
#         SGD_epoch_var.append(epoch_var/res_step)
#         epoch_loss = 0
#         epoch_var = 0
        
#     idx = np.random.randint(X.shape[0])
#     inputs = torch.tensor(X[idx].reshape(1,3,32,32)).to(device)
#     labels = torch.tensor(y, dtype=torch.long)[idx].view(1).to(device)
#     outputs = model_SGD.forward(inputs)
#     loss = criterion(outputs, labels)
#     loss.backward()
#     total_loss += loss.data.item()
#     epoch_loss += loss.data.item()
#     model_SGD_losses.append(total_loss/(epoch+1))
#     optimizer_SGD.step()
#     #running var
#     for param_group in list(model_SGD.parameters()):
#         tot = (param_group.grad.data**2).sum()
#         total_var += tot
#         epoch_var += tot
#     SGD_avg_var.append(total_var.cpu().numpy()/(epoch+1))
#     #test error
#     if (epoch % res_step == 0):
#         e = compute_error(model_SGD,X_test,y_test)
#         model_SGD_test_e.append(e)
#         print("epoch: {}, loss: {}, var : {}".format(int(epoch/res_step),model_SGD_losses[-1],SGD_avg_var[-1]))
# #         np.save('ResNet18_{}_Adam_running_loss.npy'.format(data),np.asarray(model_SGD_losses))
# #         np.save('ResNet18_{}_Adam_epoch_loss.npy'.format(data),np.asarray(model_SGD_epoch_losses))
# #         np.save('ResNet18_{}_Adam_running_var.npy'.format(data),np.asarray(SGD_avg_var))
# #         np.save('ResNet18_{}_Adam_epoch_var.npy'.format(data),np.asarray(SGD_epoch_var))
# #         np.save('ResNet18_{}_Adam_test_err.npy'.format(data),np.asarray(model_SGD_test_e))
#     optimizer_SGD.zero_grad()
#     if ((epoch % N_SAMPLES == 0) and epoch != 0):
#         scheduler_SGD.step()   
#         for param_group in optimizer_SGD.param_groups:
#             print("lr: {}",param_group['lr'])      
# print('Adam Elapsed time: {:.2f}s'.format(time.time() - tstart))
# np.save('ResNet18_{}_Adam_running_loss.npy'.format(data),np.asarray(model_SGD_losses))
# np.save('ResNet18_{}_Adam_epoch_loss.npy'.format(data),np.asarray(model_SGD_epoch_losses))
# np.save('ResNet18_{}_Adam_running_var.npy'.format(data),np.asarray(SGD_avg_var))
# np.save('ResNet18_{}_Adam_epoch_var.npy'.format(data),np.asarray(SGD_epoch_var))
# np.save('ResNet18_{}_Adam_test_err.npy'.format(data),np.asarray(model_SGD_test_e))

In [ ]:
#train SAGA
# tstart = time.time()
# total_var = 0
# epoch_var = 0
# total_loss = 0
# epoch_loss = 0
# for epoch in range(n_epochs):
#     #epoch loss and var
#     if ((epoch % res_step == 0) and (epoch != 0)):
#         model_SAGA_epoch_losses.append(epoch_loss/res_step)
#         SAGA_epoch_var.append(epoch_var/res_step)
#         epoch_loss = 0
#         epoch_var = 0
    
#     idx = np.random.randint(X.shape[0])
#     inputs = torch.tensor(X[idx].reshape(1,3,32,32)).to(device)
#     labels = torch.tensor(y, dtype=torch.long)[idx].view(1).to(device)
#     outputs = model_SAGA.forward(inputs)
#     loss = criterion(outputs, labels)
#     loss.backward()
#     total_loss += loss.data.item()
#     epoch_loss += loss.data.item()
#     model_SAGA_losses.append(total_loss/(epoch+1))
#     _,var = optimizer_SAGA.step(labels)
#     total_var += var
#     epoch_var += var
#     SAGA_avg_var.append(total_var.cpu().numpy()/(epoch+1))
#     if (epoch % res_step == 0):
#         e = compute_error(model_SAGA,X_test,y_test)
#         model_SAGA_test_e.append(e)
#         print("epoch: {}, loss: {}, var : {}".format(int(epoch/res_step),model_SAGA_losses[-1],SAGA_avg_var[-1]))
# #         np.save('ResNet18_{}_CSAGA_Adam_running_loss.npy'.format(data),np.asarray(model_SAGA_losses))
# #         np.save('ResNet18_{}_CSAGA_Adam_epoch_loss.npy'.format(data),np.asarray(model_SAGA_epoch_losses))
# #         np.save('ResNet18_{}_CSAGA_Adam_running_var.npy'.format(data),np.asarray(SAGA_avg_var))
# #         np.save('ResNet18_{}_CSAGA_Adam_epoch_var.npy'.format(data),np.asarray(SAGA_epoch_var))
# #         np.save('ResNet18_{}_CSAGA_Adam_test_err.npy'.format(data),np.asarray(model_SAGA_test_e))
#     optimizer_SAGA.zero_grad()
#     if ((epoch % N_SAMPLES == 0) and epoch != 0):
#         scheduler_SAGA.step() 
#         for param_group in optimizer_SAGA.param_groups:
#             print("lr: {}",param_group['lr'])
# print('CSAGA Adam Elapsed time: {:.2f}s'.format(time.time() - tstart))
# np.save('ResNet18_{}_CSAGA_Adam_no_beta_running_loss.npy'.format(data),np.asarray(model_SAGA_losses))
# np.save('ResNet18_{}_CSAGA_Adam_no_beta_epoch_loss.npy'.format(data),np.asarray(model_SAGA_epoch_losses))
# np.save('ResNet18_{}_CSAGA_Adam_no_beta_running_var.npy'.format(data),np.asarray(SAGA_avg_var))
# np.save('ResNet18_{}_CSAGA_Adam_no_beta_epoch_var.npy'.format(data),np.asarray(SAGA_epoch_var))
# np.save('ResNet18_{}_CSAGA_Adam_no_beta_test_err.npy'.format(data),np.asarray(model_SAGA_test_e))

In [ ]:
# #train SAGA 2 entry per class
# tstart = time.time()
# total_var = 0
# epoch_var = 0
# total_loss = 0
# epoch_loss = 0
# for epoch in range(n_epochs):
#     #epoch loss and var
#     if ((epoch % res_step == 0) and (epoch != 0)):
#         model_SAGA_2_epoch_losses.append(epoch_loss/res_step)
#         SAGA_2_epoch_var.append(epoch_var/res_step)
#         epoch_loss = 0
#         epoch_var = 0
    
#     idx = np.random.randint(X.shape[0])
#     inputs = torch.tensor(X[idx].reshape(1,3,32,32)).to(device)
#     labels = torch.tensor(y, dtype=torch.long)[idx].view(1).to(device)
#     label = int(labels.item())
#     cluster = class_clusters[label].predict(X_cluster[idx].reshape(1,-1))[0]
#     saga_idx = label*4 + cluster
#     outputs = model_SAGA_2.forward(inputs)
#     loss = criterion(outputs, labels)
#     loss.backward()
#     total_loss += loss.data.item()
#     epoch_loss += loss.data.item()
#     model_SAGA_2_losses.append(total_loss/(epoch+1))
#     closure = labels
#     _,var = optimizer_SAGA_2.step(saga_idx)
#     total_var += var
#     epoch_var += var
#     SAGA_2_avg_var.append(total_var.cpu().numpy()/(epoch+1))
#     if (epoch % res_step == 0):
#         e = compute_error(model_SAGA_2,X_test,y_test)
#         model_SAGA_2_test_e.append(e)
#         print("epoch: {}, loss: {}, var: {}".format(int(epoch/res_step),model_SAGA_2_losses[-1],SAGA_2_avg_var[-1]))
# #         np.save('ResNet18_{}_CSAGA_Adam_running_loss.npy'.format(data),np.asarray(model_SAGA_losses))
# #         np.save('ResNet18_{}_CSAGA_Adam_epoch_loss.npy'.format(data),np.asarray(model_SAGA_epoch_losses))
# #         np.save('ResNet18_{}_CSAGA_Adam_running_var.npy'.format(data),np.asarray(SAGA_avg_var))
# #         np.save('ResNet18_{}_CSAGA_Adam_epoch_var.npy'.format(data),np.asarray(SAGA_epoch_var))
# #         np.save('ResNet18_{}_CSAGA_Adam_test_err.npy'.format(data),np.asarray(model_SAGA_test_e))
#     optimizer_SAGA_2.zero_grad()
#     if ((epoch % N_SAMPLES == 0) and epoch != 0):
#         scheduler_SAGA_2.step() 
#         for param_group in optimizer_SAGA_2.param_groups:
#             print("lr: {}",param_group['lr'])
# print('CSAGA Adam 2pc Elapsed time: {:.2f}s'.format(time.time() - tstart))
# np.save('ResNet18_{}_CSAGA_Adam_2_running_loss.npy'.format(data),np.asarray(model_SAGA_2_losses))
# np.save('ResNet18_{}_CSAGA_Adam_2_epoch_loss.npy'.format(data),np.asarray(model_SAGA_2_epoch_losses))
# np.save('ResNet18_{}_CSAGA_Adam_2_running_var.npy'.format(data),np.asarray(SAGA_2_avg_var))
# np.save('ResNet18_{}_CSAGA_Adam_2_epoch_var.npy'.format(data),np.asarray(SAGA_2_epoch_var))
# np.save('ResNet18_{}_CSAGA_Adam_2_test_err.npy'.format(data),np.asarray(model_SAGA_2_test_e))

In [ ]:
#train SVRG adam
tstart = time.time()
total_var = 0
epoch_var = 0
total_loss = 0
epoch_loss = 0
SVRG_batch_size = 250 #for snapshot
model_snapshot =  ResNet18(OUT_DIM).to(device)
model_snapshot_avg =  ResNet18(OUT_DIM).to(device)
snapshot_per_epoch = 10
for epoch in range(n_epochs):  
    #epoch loss and variance
    if ((epoch % res_step == 0) and (epoch != 0)):
        model_SVRG_epoch_losses.append(epoch_loss/res_step)
        SVRG_epoch_var.append(epoch_var/res_step)
        epoch_loss = 0
        epoch_var = 0
    #snapshot 
    if (epoch % (N_SAMPLES/snapshot_per_epoch) == 0):
#         model_snapshot = copy.deepcopy(model_SVRG)
#         model_snapshot_avg = copy.deepcopy(model_snapshot)
        model_snapshot.load_state_dict(model_SVRG.state_dict())
        model_snapshot_avg.load_state_dict(model_SVRG.state_dict())
        model_snapshot_avg.zero_grad()
        #compute full batch for snapshot
        for i in range(int(X.shape[0]/SVRG_batch_size)):
            inputs = torch.from_numpy(X[i*SVRG_batch_size:(i+1)*SVRG_batch_size]).to(device)
            labels = torch.tensor(y[i*SVRG_batch_size:(i+1)*SVRG_batch_size], dtype=torch.long).to(device)
            outputs = model_snapshot_avg.forward(inputs)
            loss = criterion(outputs, labels)
            loss = loss/(int(X.shape[0]/SVRG_batch_size)) #correct value for average loss
            loss.backward()
        print("Snapshot computed at iteration {}".format(epoch))
        model_snapshot.zero_grad()
        model_SVRG.zero_grad()
            
    idx = np.random.randint(X.shape[0])
    inputs = torch.tensor(X[idx].reshape(1,3,32,32)).to(device)
    labels = torch.tensor(y, dtype=torch.long)[idx].view(1).to(device)
    #snapshot grad
    outputs = model_snapshot.forward(inputs).view(1,-1)
    loss = criterion(outputs, labels)
    loss.backward()
    #current grad
    outputs = model_SVRG.forward(inputs).view(1,-1)
    loss = criterion(outputs, labels)
    loss.backward()

    total_loss += loss.data.item()
    epoch_loss += loss.data.item()
    model_SVRG_losses.append(total_loss/(epoch+1))
    _, var = optimizer_SVRG.step(snap_grad = list(model_snapshot.parameters()),
                                 snap_avg_grad = list(model_snapshot_avg.parameters()))
#     print("Iteration: {}, loss: {}, var: {}, sample: {}".format(epoch,loss.data.item(),var,idx))
    total_var += var
    epoch_var += var
    SVRG_avg_var.append(total_var.cpu().numpy()/(epoch+1))

    #compute error
    if (epoch % res_step == 0):
        e = compute_error(model_SVRG,X_test,y_test)
        model_SVRG_test_e.append(e)
        print("epoch: {}, loss: {}, var : {}".format(int(epoch/res_step),model_SVRG_losses[-1],SVRG_avg_var[-1]))
    optimizer_SVRG.zero_grad()
    model_snapshot.zero_grad()
    #decay lr
    if ((epoch % N_SAMPLES == 0) and epoch != 0):
        scheduler_SVRG.step() 
        for param_group in optimizer_SVRG.param_groups:
            print("lr: {}",param_group['lr'])
print('SVRG Adam Elapsed time: {:.2f}s'.format(time.time() - tstart))
np.save('ResNet18_{}_SVRG_Adam_running_loss.npy'.format(data),np.asarray(model_SAGA_2_losses))
np.save('ResNet18_{}_SVRG_Adam_epoch_loss.npy'.format(data),np.asarray(model_SAGA_2_epoch_losses))
np.save('ResNet18_{}_SVRG_Adam_running_var.npy'.format(data),np.asarray(SAGA_2_avg_var))
np.save('ResNet18_{}_SVRG_Adam_epoch_var.npy'.format(data),np.asarray(SAGA_2_epoch_var))
np.save('ResNet18_{}_SVRG_Adam_test_err.npy'.format(data),np.asarray(model_SAGA_2_test_e))

In [ ]:
model_SGD_losses = np.load('ResNet18_{}_Adam_running_loss.npy'.format(data),allow_pickle = True).tolist()
model_SGD_epoch_losses = np.load('ResNet18_{}_Adam_epoch_loss.npy'.format(data),allow_pickle = True).tolist()
SGD_avg_var = np.load('ResNet18_{}_Adam_running_var.npy'.format(data),allow_pickle = True).tolist()
SGD_epoch_var = np.load('ResNet18_{}_Adam_epoch_var.npy'.format(data),allow_pickle = True).tolist()
model_SGD_test_e = np.load('ResNet18_{}_Adam_test_err.npy'.format(data),allow_pickle = True).tolist()

model_SAGA_losses = np.load('ResNet18_{}_CSAGA_Adam_running_loss.npy'.format(data),allow_pickle = True).tolist()
model_SAGA_epoch_losses = np.load('ResNet18_{}_CSAGA_Adam_epoch_loss.npy'.format(data),allow_pickle = True).tolist()
SAGA_avg_var = np.load('ResNet18_{}_CSAGA_Adam_running_var.npy'.format(data),allow_pickle = True).tolist()
SAGA_epoch_var = np.load('ResNet18_{}_CSAGA_Adam_epoch_var.npy'.format(data),allow_pickle = True).tolist()
model_SAGA_test_e = np.load('ResNet18_{}_CSAGA_Adam_test_err.npy'.format(data),allow_pickle = True).tolist()

model_SAGA_losses = np.load('ResNet18_{}_CSAGA_Adam_no_beta_running_loss.npy'.format(data),allow_pickle = True).tolist()
model_SAGA_epoch_losses = np.load('ResNet18_{}_CSAGA_Adam_no_beta_epoch_loss.npy'.format(data),allow_pickle = True).tolist()
SAGA_avg_var = np.load('ResNet18_{}_CSAGA_Adam_no_beta_running_var.npy'.format(data),allow_pickle = True).tolist()
SAGA_epoch_var = np.load('ResNet18_{}_CSAGA_Adam_no_beta_epoch_var.npy'.format(data),allow_pickle = True).tolist()
model_SAGA_test_e = np.load('ResNet18_{}_CSAGA_Adam_no_beta_test_err.npy'.format(data),allow_pickle = True).tolist()

model_SAGA_2_losses = np.load('ResNet18_{}_CSAGA_Adam_2_running_loss.npy'.format(data),allow_pickle = True).tolist()
model_SAGA_2_epoch_losses = np.load('ResNet18_{}_CSAGA_Adam_2_epoch_loss.npy'.format(data),allow_pickle = True).tolist()
SAGA_2_avg_var = np.load('ResNet18_{}_CSAGA_Adam_2_running_var.npy'.format(data),allow_pickle = True).tolist()
SAGA_2_epoch_var = np.load('ResNet18_{}_CSAGA_Adam_2_epoch_var.npy'.format(data),allow_pickle = True).tolist()
model_SAGA_2_test_e = np.load('ResNet18_{}_CSAGA_Adam_2_test_err.npy'.format(data),allow_pickle = True).tolist()

model_SVRG_losses = np.load('ResNet18_{}_SVRG_Adam_running_loss.npy'.format(data),allow_pickle = True).tolist()
model_SVRG_epoch_losses = np.load('ResNet18_{}_SVRG_Adam_epoch_loss.npy'.format(data),allow_pickle = True).tolist()
SVRG_avg_var = np.load('ResNet18_{}_SVRG_Adam_running_var.npy'.format(data),allow_pickle = True).tolist()
SVRG_epoch_var = np.load('ResNet18_{}_SVRG_Adam_epoch_var.npy'.format(data),allow_pickle = True).tolist()
model_SVRG_test_e = np.load('ResNet18_{}_SVRG_Adam_test_err.npy'.format(data),allow_pickle = True).tolist()

In [ ]:
def plot_val(losses, labels,val = "loss"):
    plt.figure(figsize=(15,5))
    for loss, label in zip(losses, labels):
        if len(loss) != 0:
            print(label,loss[-1])
        plt.plot(loss, label = label)
    plt.legend(loc='upper right')
    plt.title('ResNet cifar10 {} (class,{}) (lr: {}, n_samples = {}, betas: {})'.format(centered,
                                                                                          opti,lr,N_SAMPLES,betas))
    #plt.ylim(0,0.4)
    #plt.yscale('log')
    plt.xlabel('iteration')
    plt.ylabel(val)

In [ ]:
#running average loss
plot_val([model_SGD_losses,model_SAGA_losses,model_SAGA_2_losses,model_SVRG_losses],
         ['SGD','SAGA (gradient momentum ({}))'.format(MOMENTUM_SAGA),'SAGA 2pc','SVRG'])

In [ ]:
#average loss per epoch
plot_val([model_SGD_epoch_losses,model_SAGA_epoch_losses,model_SAGA_2_epoch_losses,model_SVRG_epoch_losses],
         ['SGD','SAGA (gradient momentum ({}))'.format(MOMENTUM_SAGA),'SAGA 2pc','SVRG'])

In [ ]:
def plot_val(losses, labels,val = "loss"):
    plt.figure(figsize=(15,5))
    for losses, label in zip(losses, labels):
        plt.plot(losses, label = label)
    plt.legend(loc='upper right')
    plt.title('ResNet cifar10 {} (class,{}) (lr: {}, n_samples = {}, betas: {})'.format(centered,
                                                                                          opti,lr,N_SAMPLES,betas))
    plt.xlabel('iteration')
    #plt.ylim(0,100)
    plt.ylabel(val)

In [ ]:
#running var
plot_val([SGD_avg_var,SAGA_avg_var,SAGA_2_avg_var,SVRG_avg_var],
         ['SGD var','SAGA var (gradient momentum ({}))'.format(MOMENTUM_SAGA),'SAGA 2pc','SVRG'],"variance")

In [ ]:
#var at each epoch
plot_val([SGD_epoch_var,SAGA_epoch_var,SAGA_2_epoch_var,SVRG_epoch_var],
         ['SGD var','SAGA var (gradient momentum ({}))'.format(MOMENTUM_SAGA),'SAGA 2pc','SVRG'],"variance")

In [ ]:
def plot_val(losses, labels,val = "loss"):
    plt.figure(figsize=(15,5))
    for loss, label in zip(losses, labels):
        if len(loss) != 0:
            print(label,loss[-1])
        plt.plot(loss, label = label)
    plt.legend(loc='upper right')
    plt.title('ResNet cifar10 {} (class,{}) (lr: {}, n_samples = {}, betas: {})'.format(centered,
                                                                                          opti,lr,N_SAMPLES,betas))
    plt.xlabel('iteration')
    #plt.ylim(0.4,1.1)
    plt.ylabel(val)

In [ ]:
s = np.array(SAGA_avg_var)
t = np.array(SGD_avg_var)
s2 = np.array(SAGA_2_avg_var)
ratio1 = s/t
#ratio2 = s2/t
#type(s[0])
plot_val([ratio1],["ratio saga"],val = "var ratio")

In [ ]:
def plot_err(losses, labels,val = "Error"):
    plt.figure(figsize=(6,6))
    for losses, label in zip(losses, labels):
        plt.plot(losses, label = label)
    plt.legend(loc='upper right')
    plt.xlabel('Iteration (k)',fontsize = 25)
    plt.xticks(fontsize=13)
    plt.yticks(fontsize=13)
    plt.ylabel(val,fontsize = 25)
    #plt.ylim(50, 300)

In [ ]:
plot_err([model_SGD_test_e,model_SAGA_test_e,model_SAGA_2_test_e,model_SVRG_test_e],
         ['SGD','SAGApc(1)','SAGApc(2)','SVRG'])

In [ ]:
# from torchsummary import summary
# summary(model_SAGA, input_size=(3,32,32))